# Code generator

Neural network that generates Golang code

In [1]:
import json
import numpy as np
import os
import tensorflow as tf
import time

In [2]:
class Dataset(object):
    dataset_file = 'dataset.dt'
    dataset_folder = 'dataset'
    
    def __init__(self):
        dataset = self.load_dataset()
        self.encoded, self.vocab_to_int, self.int_to_vocab = dataset
        
    def load_dataset(self):
        if self.exists_clean_dataset():
            return self.load_clean_dataset()
        dataset = self.create_dataset()
        self.save_clean_dataset(dataset)
        return dataset

    def exists_clean_dataset(self):
        return os.path.isfile(self.dataset_file)

    def load_clean_dataset(self):
        with open(self.dataset_file, "r") as d:
            dataset = json.loads(d.read())
            return (
                np.array(dataset["encoded"]),
                dataset["vocab_to_int"],
                dataset["int_to_vocab"]
            )
        
    def create_dataset(self):
        file_names = self.get_file_names()
        content = self.get_content_from_files(file_names)
        return self.create_vocab(content)
            
    def get_file_names(self):
        files = os.listdir(self.dataset_folder)
        return [f for f in files if f.endswith('.go')]
    
    def get_content_from_files(self, file_names):
        content = ''
        for name in file_names:
            with open(os.path.join(self.dataset_folder, name), 'r') as f:
                content += f.read() + '\n'
        return content
    
    def create_vocab(self, content):
        vocab = set(content)
        vocab_to_int = {c: i for i, c in enumerate(vocab)}
        int_to_vocab = dict(enumerate(vocab))
        encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
        return encoded, vocab_to_int, int_to_vocab
    
    def save_clean_dataset(self, dataset):
        encoded, v_to_int, int_to_v = dataset
        with open(self.dataset_file, "w+") as d:
            d.write(json.dumps({
                'encoded' : [int(i) for i in encoded], 
                'vocab_to_int' : v_to_int,
                'int_to_vocab' : int_to_v
            }))

In [9]:
class Model(object):
    batch_size = None
    sequence_size = None
    
    hidden_layer_size = 512
    num_layers = 2
    
    learning_rate = 0.0005
    
    checkpoint = None
    checkpoints_folder = 'checkpoints'
    
    def __init__(self):
        self.init()
        self.build_graph()
        self.init_graph()
    
    def init(self):
        tf.reset_default_graph()
        self.dataset = Dataset()
        self.io_size = len(self.dataset.vocab_to_int)
        self.create_checkpoints_folder()
        
    def create_checkpoints_folder(self):
        if not os.path.exists(self.checkpoints_folder):
            os.makedirs(self.checkpoints_folder)

    def build_graph(self):
        self.build_inputs()
        lstm_outputs = self.build_lstm()
        logits = self.build_output(lstm_outputs)
        self.build_loss(logits)
        self.build_optimizer()
        
    def build_inputs(self):
        self.inputs = tf.placeholder(tf.int32, [None, self.sequence_size], name='inputs')
        self.targets = tf.placeholder(tf.int32, [None, self.sequence_size], name='targets')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
    def build_lstm(self):
        def create_lstm():
            lstm = tf.contrib.rnn.BasicLSTMCell(self.hidden_layer_size)
            lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=self.keep_prob)
            return lstm
        
        cell = tf.contrib.rnn.MultiRNNCell([create_lstm() for _ in range(self.num_layers)])
        self.initial_state = cell.zero_state(self.batch_size, tf.float32)
        x_one_hot = tf.one_hot(self.inputs, self.io_size)
        outputs, self.final_state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        return outputs
    
    def build_output(self, lstm_outputs):
        x = tf.reshape(lstm_outputs, [-1, self.hidden_layer_size])
        
        weights = tf.Variable(tf.truncated_normal((self.hidden_layer_size, self.io_size), stddev=0.1))
        bias = tf.Variable(tf.zeros(self.io_size))
        
        logits = tf.add(tf.matmul(x, weights), bias)
        
        self.prediction = tf.nn.softmax(logits)
        
        return logits
    
    def build_loss(self, logits):
        y_one_hot = tf.one_hot(self.targets, self.io_size)
        y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
        self.loss = tf.reduce_mean(loss)
        
    def build_optimizer(self):
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.optimizer = optimizer.minimize(self.loss)
    
    def init_graph(self):
        self.saver = tf.train.Saver()
        self.session = tf.Session()
        init = tf.global_variables_initializer()
        self.session.run(init)
        self.restore_graph()
        
    def restore_graph(self):
        if self.checkpoint:
            checkpoint_path = '%s/%s.ckpt' % (self.checkpoints_folder, self.checkpoint)
            self.saver.restore(self.session, checkpoint_path)

In [4]:
class Trainer(Model):
    batch_size = 200
    sequence_size = 100
    
    KEEP_PROB = 0.5
    
    def __init__(self):
        Model.__init__(self)
        self._init()
        self.create_batches()
        
    def _init(self):
        self.batch_inputs = []
        self.batch_targets = []
        
        bs = self.batch_size * self.sequence_size
        n_batches = len(self.dataset.encoded) // bs
        self.dataset.encoded = self.dataset.encoded[:n_batches * bs]
        self.dataset.encoded = self.dataset.encoded.reshape(
            (self.batch_size), -1
        )
        
    def create_batches(self):
        encoded = self.dataset.encoded
        for n in range(0, encoded.shape[1], self.sequence_size):
            x = encoded[:, n:n + self.sequence_size]
            y = np.zeros_like(x)
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
            
            self.batch_inputs.append(x)
            self.batch_targets.append(y)
            
            
    def train(self, epochs):
        for epoch in range(epochs):
            self.train_batches(epoch)
            self.saver.save(
                self.session, '%s/model_epoch_%d.ckpt' % (self.checkpoints_folder, epoch + 1))
        self.saver.save(
                self.session, '%s/model_trained.ckpt' % self.checkpoints_folder)

    def train_batches(self, epoch):
        n_state = self.session.run(self.initial_state)
        for inputs, targets in zip(self.batch_inputs, self.batch_targets):
            t = time.time()
            batch_loss, n_state = self.optimize_model(inputs, targets, n_state)
            print(
                'Epoch : %s Training loss : %2f, sec/batch : %2f' %
                (epoch, batch_loss, (time.time() - t))
            )
    
    def optimize_model(self, x, y, state):
        batch_loss, n_state, _ = self.session.run(
            [self.loss, self.final_state, self.optimizer],
            feed_dict={
                self.inputs: x,
                self.targets: y,
                self.keep_prob: self.KEEP_PROB,
                self.initial_state: state
            }
        )
        return batch_loss, n_state

Epoch : 0 Training loss : 6.247478, sec/batch : 14.830582
Epoch : 0 Training loss : 6.187508, sec/batch : 14.941288
Epoch : 0 Training loss : 6.091173, sec/batch : 14.375744
Epoch : 0 Training loss : 5.841241, sec/batch : 14.237627
Epoch : 0 Training loss : 5.271130, sec/batch : 13.795021
Epoch : 0 Training loss : 5.618031, sec/batch : 14.101972
Epoch : 0 Training loss : 4.902487, sec/batch : 15.014810
Epoch : 0 Training loss : 4.980622, sec/batch : 14.165095
Epoch : 0 Training loss : 4.960245, sec/batch : 13.869857
Epoch : 0 Training loss : 4.730867, sec/batch : 14.027809
Epoch : 0 Training loss : 4.503460, sec/batch : 13.897916
Epoch : 0 Training loss : 4.336017, sec/batch : 13.977636
Epoch : 0 Training loss : 4.242531, sec/batch : 13.997731
Epoch : 0 Training loss : 4.202864, sec/batch : 13.901595
Epoch : 0 Training loss : 4.193212, sec/batch : 13.896856
Epoch : 0 Training loss : 4.091153, sec/batch : 14.051506
Epoch : 0 Training loss : 4.059059, sec/batch : 14.291140
Epoch : 0 Trai

In [17]:
class Predictor(Model):
    checkpoint = 'model_trained'
    sequence_size = 1
    batch_size = 1
    
    def __init__(self):
        Model.__init__(self)
        self.code = [c for c in 'package generator']
        

    def create_code(self, size):
        previous_state, x, c = self.generate_from_base_code()
        return self.generate_new_code(previous_state, x, c, size)
        
    def generate_from_base_code(self):
        previous_state = self.session.run(self.initial_state)
        for c in self.code:
            x = np.array([[self.dataset.vocab_to_int[c]]])
            softmax, previous_state = self.session.run(
                [self.prediction, self.final_state],
                feed_dict={
                    self.inputs: x,
                    self.keep_prob: 1.0,
                    self.initial_state: previous_state
                }
            )
        return previous_state, x, self.add_to_code(softmax)
    
    def generate_new_code(self, previous_state, x, c, size):
        for i in range(size):
            x[0, 0] = c
            softmax, previous_state = self.session.run(
                [self.prediction, self.final_state],
                feed_dict={
                    self.inputs: x,
                    self.keep_prob: 1.0,
                    self.initial_state: previous_state
                }
            )
            c = self.add_to_code(softmax)
        return ''.join(self.code)
    
    def add_to_code(self, softmax):
        softmax = np.squeeze(softmax)
        softmax[np.argsort(softmax)[:-4]] = 0
        softmax /= np.sum(softmax)
        c = np.random.choice(len(self.dataset.int_to_vocab), 1, p=softmax)[0]
        self.code.append(self.dataset.int_to_vocab[str(c)])
        return c

checkpoints/model_trained.ckpt
package generator   
  t 
 
  t 	                                       
                              0              


In [24]:
if os.path.isfile('checkpoints/model_trained.ckpt.meta'):
    print('Prediction:\n\n')
    print Predictor().create_code(1000)
else:
    print('Training...\n\n')
    Trainer().train(10)

training
